In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from PIL import Image

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_generator = ImageDataGenerator(rescale = 1./255, validation_split=0.2)

In [ ]:
height = 200
width = 200

In [ ]:
train_generator = data_generator.flow_from_directory(
    directory = "/content/drive/MyDrive/유방암 예측 data/train_imgs",
    target_size = (height,width),
    batch_size = 64,
    class_mode = 'binary',
    subset = 'training',
    shuffle = False)

Found 801 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'notransfer': 0, 'transfer': 1}

In [ ]:
val_generator = data_generator.flow_from_directory(
    directory = "/content/drive/MyDrive/유방암 예측 data/train_imgs",
    target_size =(height, width),
    class_mode = 'binary',
    batch_size = 64,
    subset='validation',
    shuffle = False)

Found 199 images belonging to 2 classes.


## 모델 구성

In [ ]:
model = Sequential()
# 입력층
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', input_shape=(height, width, 3)))
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size = (2,2),strides = 1))

model.add(Conv2D(filters=16, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='valid', activation='relu'))

model.add(MaxPool2D(pool_size = (2,2), strides = 1))
model.add(Dropout(0.2))
# Fully-connected layer
model.add(Flatten())
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))

# 출력층 - binary classification
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 200, 200, 16)      448       
                                                                 
 conv2d_5 (Conv2D)           (None, 198, 198, 16)      2320      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 197, 197, 16)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 195, 195, 16)      2320      
                                                                 
 conv2d_7 (Conv2D)           (None, 193, 193, 16)      2320      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 192, 192, 16)     0         
 2D)                                                  

In [ ]:
# compile
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics ='accuracy')

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
# fit
model.fit(train_generator, epochs = 1000, callbacks = early_stop, validation_data = val_generator)

KeyboardInterrupt: ignored